In [1]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from copy import deepcopy
import scipy
np.random.seed(1)

In [2]:

# print(bp104_1)
# print(bp104_2)
# print(bp104_3)
# print(bp104_4)

#             for i in range (len(measurments)):
#                 like_ = scipy.stats.norm.pdf(z_t[i], measurments[i][0].flatten(), measurments[i][1].flatten())
#                 likelihood*=like_

In [13]:
'''
create_singular_model: if True will creaet 
'''
class MeasurmentModel:
    def __init__(self, map_, bp104s, normed_consts, var_const = 1, create_singular_model = False):
        self.map_ = map_
        self.bp104_1, self.bp104_2, self.bp104_3, self.bp104_3 = bp104s 
        self.normed_consts = normed_consts
        self.gp_models = None
        self.var_const = var_const
        self.kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))
        self.create_singular_model = create_singular_model
        self.bp104_1_loc = [0.5, 2.5]
        self.bp104_2_loc = [16.5,9.5]
        self.bp104_3_loc = [18.5,2.5]
        self.bp104_4_loc = [10.5,4.5]
        
        if self.create_singular_model:
            # create one model for all beacons
            print("Creating one gp model for all beacons.")
            self.gp_models = self._create_singular_gp_model()
        else:
            # Create four models - one per beacon
            print("Creating four gp model for each beacons.")
            self.gp_models = []
            self._create_four_gp_models()
            
    """
    Function gets liklihood of observing z_t at state_t based on the 
    GP regression model 
    z_t: actual measurment at time t
    state_t: state of the particle at time t
    """
    def get_likelihood(self, z_t, state_t):
        if self.create_singular_model:
            mu, sigma = self.predict(state_t)
            l1, l2, l3, l4 = scipy.stats.norm.pdf(z_t, mu.flatten(), sigma.flatten())
            likelihood = l1 * l2 * l3 * l4
            if likelihood == 0:
                return 1.e-300
            else:
                print('final_likelihood', likelihood)
                return likelihood
        else:
            measurments = self.predict(state_t)
            likelihood = 1.0
            likelihood_ = scipy.stats.norm.pdf(z_t, measurments[:,0], np.diag(measurments[:,1]))
            for i in range (likelihood_.shape[0]):
                likelihood = likelihood *  likelihood_[i,i]
            if likelihood == 0:
                return 1.e-300 # avoid norm. problems
            else:
                return likelihood
        
    def predict(self, state):
        if self.create_singular_model:
            mu, sigma = self.gp_models.predict([[state[0], state[1]]], return_cov=True)
            return [mu, sigma]
        else:
            measurment = []
            for i in range(len(self.gp_models)):
                mu, sigma = self.gp_models[i].predict([[state[0], state[1]]], return_cov=True)
                measurment.append(np.array([mu.flatten()[0], sigma.flatten()[0]]))
            return np.array(measurment)

    def _get_beacon_dists(self, state):
        bp104_1_dist = np.sqrt((state[0]-self.bp104_1_loc[0])**2 + (state[1] - self.bp104_1_loc[1])**2)
        bp104_2_dist = np.sqrt((state[0]-self.bp104_2_loc[0])**2 + (state[1] - self.bp104_2_loc[1])**2)
        bp104_3_dist = np.sqrt((state[0]-self.bp104_3_loc[0])**2 + (state[1] - self.bp104_3_loc[1])**2)
        bp104_4_dist = np.sqrt((state[0]-self.bp104_4_loc[0])**2 + (state[1] - self.bp104_4_loc[1])**2)

        dist_arr = np.array([bp104_1_dist, bp104_2_dist, bp104_3_dist, bp104_4_dist])
        dist_arr = 1./dist_arr
        normed_dist_arr = dist_arr / np.sum(dist_arr)
        return normed_dist_arr
    
    def _create_four_gp_models(self):
        self.gp_models.append(self._create_one_beacon_model(bp104_1))
        self.gp_models.append(self._create_one_beacon_model(bp104_2))
        self.gp_models.append(self._create_one_beacon_model(bp104_3))
        self.gp_models.append(self._create_one_beacon_model(bp104_4))
        print("Created 4 GP models")
        
    def _create_singular_gp_model(self):
        for i in range (bp104_1.shape[0]):
            for j in range (bp104_1.shape[1]):    
                X.append(np.array([i,j])) # just the state [0,0], [0,1] etc... 
                y.append(np.array([bp104_1[i,j], bp104_2[i,j], bp104_3[i,j], bp104_4[i,j]]))
        X= np.array(X)
        y= np.array(y)
        dy = np.random.random(y.shape) * self.var_const
        # Instantiate a Gaussian Process model
        gp = GaussianProcessRegressor(kernel=self.kernel,normalize_y= True, alpha = dy,
                                      n_restarts_optimizer=10)
        
        # Fit to data using Maximum Likelihood Estimation of the parameters
        gp.fit(X,y)
        return gp
    
    def _create_one_beacon_model(self, bp104_x):
        X = []
        y = []
        for i in range (bp104_x.shape[0]):
            for j in range (bp104_x.shape[1]):    
                X.append(np.array([i,j])) # just the state [0,0], [0,1] etc... 
                y.append(bp104_x[i,j])

        X= np.array(X)
        y= np.array(y)
        dy = np.random.random(y.shape)* self.var_const
        # Instantiate a Gaussian Process model
        gp_bp104x = GaussianProcessRegressor(kernel=self.kernel, alpha=dy,normalize_y= True,
                                      n_restarts_optimizer=10)

        # Fit to data using Maximum Likelihood Estimation of the parameters
        gp_bp104x.fit(X,y)
        return gp_bp104x 
    
    def get_normalized_values(self, measurment):
        #measurment should be four values 
        return np.array(measurment) - np.array(self.normed_consts)
    
    def return_gp_models(self):
        return self.gp_models

In [14]:
map_ = np.load("/Users/kushagratiwary/Documents/ECE445/map_data/final_041719_0112/map_binary_.npy")
bp104_1 = np.load("/Users/kushagratiwary/Documents/ECE445/map_data/final_041719_0112/bp104_1.npy")
bp104_2 = np.load("/Users/kushagratiwary/Documents/ECE445/map_data/final_041719_0112/bp104_2.npy")
bp104_3 = np.load("/Users/kushagratiwary/Documents/ECE445/map_data/final_041719_0112/bp104_3.npy")
bp104_4 = np.load("/Users/kushagratiwary/Documents/ECE445/map_data/final_041719_0112/bp104_4.npy")
norm_constants = np.load("/Users/kushagratiwary/Documents/ECE445/map_data/final_041719_0112/norm_constants.npy")


measurments = [
    [-51.,-80.,-66.,-64.],
    [-53.,-80.,-67.,-62.],

    [-59.,-80.,-67.,-62.],
    [-57.,-80.,-64.,-64],
    
    [-58.,-66.,-61.,-52.],
    [-58.,-63.,-63.,-52.],
    
    [-65.,-59.,-56.,-60.],
    [-63.,-62.,-61.,-59.],
    
    [-66.,-62.,-52.,-61.],
    [-65.,-58.,-52.,-61.],
    
    [-80.,-52.,-54.,-66.],
    [-80.,-51.,-55.,-66.],

    [-67.,-57.,-58.,-64.],
    [-68.,-60.,-57.,-62.]
]

simulated_measurments = np.array(measurments).astype(float)
print(norm_constants)
np.seterr(divide='ignore', invalid='ignore')

[-41 -44 -44 -46]


{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [15]:
GP_measurment_model = MeasurmentModel(map_, np.array([bp104_1, bp104_2, bp104_3, bp104_4]), 
                                      norm_constants, var_const = 3)

Creating four gp model for each beacons.
Created 4 GP models


In [16]:
gp_models = GP_measurment_model.return_gp_models()

In [186]:
path = "/Users/kushagratiwary/Documents/ECE445/_gp_approx_data/final_/map1/_x1015_y0510/"

In [187]:
_x05y05_b1_mean = np.zeros((50,50))
_x05y05_b2_mean = np.zeros((50,50))
_x05y05_b3_mean = np.zeros((50,50))
_x05y05_b4_mean = np.zeros((50,50))

_x05y05_b1_cov = np.zeros((50,50))
_x05y05_b2_cov = np.zeros((50,50))
_x05y05_b3_cov = np.zeros((50,50))
_x05y05_b4_cov = np.zeros((50,50))

for i in range (10,15):
    for j in range (5,10):
        print('----------i,j--------', i,j)
        for k in range (10):
            for l in range (10):
                idx_x = int((i%5)*10 + k)
                idx_y = int((j%5)*10 + l)
#                 print('k,l', k,l)
#                 print('idx_x', idx_x)
#                 print('idx_y', idx_y)
#                 print("state:", [i+(k/10.0), j+(l/10.0)])
                mub1, cov1 = gp_models[0].predict([[i+(k/10.0), j+(l/10.0)]], return_cov=True)
                mub2, cov2 = gp_models[1].predict([[i+(k/10.0), j+(l/10.0)]], return_cov=True)
                mub3, cov3 = gp_models[2].predict([[i+(k/10.0), j+(l/10.0)]], return_cov=True)
                mub4, cov4 = gp_models[3].predict([[i+(k/10.0), j+(l/10.0)]], return_cov=True)
                _x05y05_b1_mean[idx_x, idx_y] = np.around(mub1,2)
                _x05y05_b2_mean[idx_x, idx_y] = np.around(mub2,2)
                _x05y05_b3_mean[idx_x, idx_y] = np.around(mub3,2)                
                _x05y05_b4_mean[idx_x, idx_y] = np.around(mub4,2)                

                _x05y05_b1_cov[idx_x, idx_y] = np.around(cov1,2)
                _x05y05_b2_cov[idx_x, idx_y] = np.around(cov2,2)
                _x05y05_b3_cov[idx_x, idx_y] = np.around(cov3,2)                
                _x05y05_b4_cov[idx_x, idx_y] = np.around(cov4,2)   

----------i,j-------- 10 5
----------i,j-------- 10 6
----------i,j-------- 10 7
----------i,j-------- 10 8
----------i,j-------- 10 9
----------i,j-------- 11 5
----------i,j-------- 11 6
----------i,j-------- 11 7
----------i,j-------- 11 8
----------i,j-------- 11 9
----------i,j-------- 12 5
----------i,j-------- 12 6
----------i,j-------- 12 7
----------i,j-------- 12 8
----------i,j-------- 12 9
----------i,j-------- 13 5
----------i,j-------- 13 6
----------i,j-------- 13 7
----------i,j-------- 13 8
----------i,j-------- 13 9
----------i,j-------- 14 5
----------i,j-------- 14 6
----------i,j-------- 14 7
----------i,j-------- 14 8
----------i,j-------- 14 9


In [189]:
# np.savetxt(path+'b1_mean.csv', _x05y05_b1_mean, delimiter=",", fmt='%1.2f')
# np.savetxt(path+'b2_mean.csv', _x05y05_b2_mean, delimiter=",", fmt='%1.2f')
# np.savetxt(path+'b3_mean.csv', _x05y05_b3_mean, delimiter=",", fmt='%1.2f')
# np.savetxt(path+'b4_mean.csv', _x05y05_b4_mean, delimiter=",", fmt='%1.2f')

# np.savetxt(path+'b1_cov.csv', _x05y05_b1_cov, delimiter=",", fmt='%1.2f')
# np.savetxt(path+'b2_cov.csv', _x05y05_b2_cov, delimiter=",", fmt='%1.2f')
# np.savetxt(path+'b3_cov.csv', _x05y05_b3_cov, delimiter=",", fmt='%1.2f')
# np.savetxt(path+'b4_cov.csv', _x05y05_b4_cov, delimiter=",", fmt='%1.2f')

In [160]:
# import os
# path = "/Users/kushagratiwary/Documents/ECE445/_gp_approx_data/"
# # path = path+'non_truncated/'
# all_dirs = os.listdir(path)
# for dir_ in all_dirs:
#     if (dir_[0] != '.'):
#         files = os.listdir(path+dir_)
#         for file_ in files:
#             if file_ == '.DS_Store':
#                 continue
#             print("at:", dir_)
#             file_name = os.path.join(path+dir_,file_)
#             print("Currently processing:", file_name)
#             array = genfromtxt(file_name, delimiter=',')
#             array_new = np.around(array, decimals=2)
#             new_direc = os.path.join(path+dir_+"_new_direc")
# #             print("new directory create:", new_file_name)
#             if not os.path.exists(new_direc):
#                 os.makedirs(new_direc)
# #             print("new file save used at:", os.path.join(new_direc,file_))
# #             np.savetxt(os.path.join(new_direc,file_), array_new, delimiter=",")

In [90]:
from numpy import genfromtxt
# my_data = genfromtxt('my_file.csv', delimiter=',')

In [116]:
my_data = genfromtxt('/Volumes/NO NAME/map1/_x1015_y1015/b2_cov.csv', delimiter=',')

In [118]:
import csv


In [ ]:
path = "/Volumes/NO NAME/map1/"
path2= "/Volumes/NO NAME/map2/"

all_dirs = os.listdir(path)
for dir_ in all_dirs:
    if (dir_[0] != '.'):
        files = os.listdir(path+dir_)
        for file_ in files:
            if file_ == '.DS_Store':
                continue
            print("at:", dir_)
            file_name = os.path.join(path+dir_,file_)
            print("Currently processing:", file_name)
            array = genfromtxt(file_name, delimiter=',')
            new_save_dir = os.path.join(path2,dir_)
            if not os.path.exists(new_save_dir):
                os.makedirs(new_save_dir)
            

In [119]:
with open('/Volumes/NO NAME/map2/asd.csv','wb') as fp:
    a = csv.writer(fp,delimiter = ",")
    a.writerows(map(lambda t: (t[0], "%.2f" % t[1]), my_data))

TypeError: a bytes-like object is required, not 'str'

In [157]:
gp_models[0].predict([[0.9, 4.10]], return_cov=True)

(array([-5.82965482]), array([[1.95513181]]))

In [158]:
gp_models[1].predict([[0.9560690629980964, 4.103558298359111]], return_cov=True)

(array([-35.71507521]), array([[0.96232942]]))

In [159]:
gp_models[2].predict([[0.9560690629980964, 4.103558298359111]], return_cov=True)

(array([-19.84738947]), array([[2.42204413]]))

In [140]:
gp_models[3].predict([[0.9560690629980964, 4.103558298359111]], return_cov=True)

(array([-11.39007724]), array([[2.79656999]]))

In [154]:
pt = "/Users/kushagratiwary/Documents/ECE445/_gp_approx_data/final_/map1/_x05_y05/b1_mean.csv"
# pt = "/Users/kushagratiwary/Documents/ECE445/_gp_approx_data/final_/map1/_x05_y05/b1_cov.csv"

my_data = genfromtxt(pt, delimiter=',')

In [156]:
my_data[9,41]
# np.where(my_data == 1.583)

-5.83